In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time 

In [3]:
def get_html(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

In [18]:
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=14&sc=14204&sngz=&po1=25&pc=50&page={}"
all_data = []
page = 0

# get html
# 1ページ目のみの例
for i in range(1):
    page += 1
    url = base_url.format(page)
    soup = get_html(url)
    # extract all tbody
    tbodys = soup.findAll("tbody")
    url_list =[]
    # （１ページ目の）詳細ページの全urlを取得
    for tbody in tbodys:
        url = "https://suumo.jp" + tbody.findAll("td")[8].find("a").get("href")
        url_list.append(url)

for url in url_list:
    base_data = {}
    soup_2 = get_html(url)
    # 部屋ごとの詳細情報を取得し、リスト化
    name = soup_2.find('h1', class_='section_h1-header-title')
    base_data["名称"] = name.text
    stations = soup_2.find("table", class_="property_view_table").findAll("tr")[1].findAll("div")
    station_list=[]
    for station in stations:
        station_list.append(station.text)
    base_data["アクセス"] = station_list
    table_1 = soup_2.find('table', class_='property_view_table')
    base_data["向き"] = table_1.findAll("tr")[4].findAll("td")[0].getText().strip()
    table_2 = soup_2.find('table', class_='data_table table_gaiyou')
    base_data["間取り詳細"] = table_2.findAll("tr")[0].findAll("td")[0].getText().strip()
    base_data["構造体素材"] = table_2.findAll("tr")[0].findAll("td")[1].getText().strip()
    all_data.append(base_data)
    # 本当はサーバーに負荷をかけないようにsleepでリクエストの時間を開けた方が良い
    time.sleep(1)

# convert to dataframe
df = pd.DataFrame(all_data)

In [19]:
df

,名称,アクセス,向き,間取り詳細,構造体素材
0,マストクレリアン鎌倉 605号室,"[ＪＲ横須賀線/大船駅 歩13分, 湘南モノレール/富士見町駅 歩16分, ＪＲ横須賀線/北...",南,洋6.9 LDK12 納戸 4.2,鉄骨
1,リブリ楓,"[ＪＲ東海道本線/大船駅 歩9分, 湘南モノレール/富士見町駅 歩3分, 湘南モノレール/湘...",南西,-,木造
2,リブリ・楓,"[ＪＲ東海道本線/大船駅 歩10分, 湘南モノレール/富士見町駅 歩4分]",西,洋8.5,木造
3,リブリ・楓,"[ＪＲ東海道本線/大船駅 歩10分, 湘南モノレール/富士見町駅 歩4分]",南西,洋8.5,木造
4,リブリ・楓,"[ＪＲ東海道本線/大船駅 歩9分, 湘南モノレール/富士見町駅 歩3分, 湘南モノレール/湘...",南西,洋8.5 K2.0,木造
...,...,...,...,...,...
75,リブリ・江ノ島II,"[江ノ島電鉄線/腰越駅 歩10分, 湘南モノレール/片瀬山駅 歩9分, 湘南モノレール/目白...",南西,洋8.4,鉄骨
76,リブリ・江ノ島II,"[湘南モノレール/片瀬山駅 歩8分, 江ノ島電鉄線/腰越駅 歩9分, ＪＲ東海道本線/藤沢駅...",-,-,鉄骨
77,リブリ・江ノ島II 301号室,"[湘南モノレール/片瀬山駅 歩8分, 江ノ島電鉄線/腰越駅 歩9分, ＪＲ東海道本線/藤沢駅...",-,-,鉄骨
78,江ノ島電鉄線 腰越駅 3階建 築5年,"[江ノ島電鉄線/腰越駅 歩9分, 湘南モノレール/片瀬山駅 歩8分]",南西,洋8.4,鉄骨


In [38]:
df.to_csv('data.csv')#,header=False, index=False)